In [1]:
import os
import pandas as pd
import numpy as np


data_dir = "/data3/lsf/Pein/Power-Prediction/data"

train_input_path = os.path.join(data_dir, "train_input_merge.csv")
train_power_path = os.path.join(data_dir, "train_power_merge.csv")

test_input_path = os.path.join(data_dir, "test_input_merge.csv")
test_power_path = os.path.join(data_dir, "test_power_merge.csv")

In [2]:
train_input_ori = pd.read_csv(train_input_path)
train_power_ori = pd.read_csv(train_power_path)
test_input_ori = pd.read_csv(test_input_path)
test_power_ori = pd.read_csv(test_power_path)

In [3]:
# Drop the 'initial_time' column
train_input_ori = train_input_ori.drop(columns=["initial_time"])
test_input_ori = test_input_ori.drop(columns=["initial_time"])

# Merge the input and power dataframes on the 'time' column
train_data = pd.merge(train_input_ori, train_power_ori, on="time")
test_data = pd.merge(test_input_ori, test_power_ori, on="time")

### Conduct a simple clearning


In [4]:
# Ensure 'power' is non-zero, setting negative values to 0
train_data["power"] = train_data["power"].apply(lambda x: max(x, 0))
test_data["power"] = test_data["power"].apply(lambda x: max(x, 0))

In [5]:
# Check the nan values in the dataset
print(train_data.isnull().sum().sum())
print(test_data.isnull().sum().sum())

# check all the 'power' is non-negative
print(train_data[train_data["power"] < 0].shape[0])
print(test_data[test_data["power"] < 0].shape[0])

0
0
0
0


In [11]:
n_train = len(train_data["time"])
n_test = len(test_data["time"])

train_date_range = pd.date_range(
    start=train_data.time.min(), end=train_data.time.max(), freq="15T"
)

test_date_range = pd.date_range(
    start=test_data.time.min(), end=test_data.time.max(), freq="15T"
)

print(f"For train: {n_train} vs {len(train_date_range)}")
print(f"For test: {n_test} vs {len(test_date_range)}")

For train: 14592 vs 14592
For test: 2880 vs 2880


/tmp/ipykernel_1705223/1235129531.py:4: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  train_date_range = pd.date_range(
/tmp/ipykernel_1705223/1235129531.py:8: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  test_date_range = pd.date_range(


In [20]:
# Ignore 'time' and 'lead_hour' for cleaning
ignore_features = ["time", "lead_hour"]
features = [col for col in train_data.columns if col not in ignore_features + ["power"]]

In [7]:
# Select features with small variance
from sklearn.feature_selection import VarianceThreshold

# Step 1: Variance Threshold Filtering
selector = VarianceThreshold(threshold=0.1)  # Adjust the threshold as needed
train_data_variance_filtered = selector.fit_transform(train_data[features])
selected_variance_features = train_data[features].columns[selector.get_support()]

In [8]:
# Create new train and test data with selected features
train_data = train_data[
    ignore_features + selected_variance_features.tolist() + ["power"]
]
test_data = test_data[ignore_features + selected_variance_features.tolist() + ["power"]]

In [10]:
print(
    f"Selected Features has {len(selected_variance_features)}:\n",
    selected_variance_features,
)
print(f"Total num of feature is {len(train_data.columns)}")

Selected Features has 89:
 Index(['altitude', 'azimuth', 'u100', 'v100', 'fg10', 'u10', 'v10', 'u200',
       'v200', 'd2m', 't2m', 'bld', 'blh', 'cape', 'capes', 'cdir', 'deg0l',
       'degm10l', 'dsrp', 'ewss', 'fdir', 'flsr', 'gwd', 'hcc', 'hwbt0',
       'hwbt1', 'i10fg', 'ilspf', 'ishf', 'kx', 'lblt', 'lcc', 'lgws',
       'litoti', 'lmlt', 'lspf', 'ltlt', 'mcc', 'mgws', 'mlcape100',
       'mlcape50', 'mld', 'mn2t', 'msl', 'mucape', 'mudlp', 'mx2t', 'nsss',
       'parcs', 'par', 'totalx', 'skt', 'slhf', 'sp', 'sshf', 'ssrc', 'ssrdc',
       'ssrd', 'ssr', 'sst', 'stl1', 'stl2', 'stl3', 'stl4', 'strc', 'strdc',
       'strd', 'str', 'sund', 'tcc', 'tcsw', 'tcw', 'tcwv', 'tisr', 'trpp',
       'tsrc', 'tsr', 'ttrc', 'ttr', 'u10n', 'uvb', 'v10n', 'vimd', 'p3020',
       'viwve', 'viwvn', 'ws200', 'ws100', 'ws10'],
      dtype='object')
Total num of feature is 92


In [11]:
# Save the combined datasets to CSV files
train_data.to_csv(f"{data_dir}/" + "train_data_92.csv", index=False)
test_data.to_csv(f"{data_dir}/" + "test_data_92.csv", index=False)

print(
    "Combined train and test datasets have been saved to 'train_data.csv' and 'test_data.csv'."
)

Combined train and test datasets have been saved to 'train_data.csv' and 'test_data.csv'.


In [3]:
from prepare_data import WindPowerDataset

data_dir = "/data3/lsf/Pein/Power-Prediction/data"
train_ori = os.path.join(data_dir, "train_data.csv")
test_ori = os.path.join(data_dir, "test_data.csv")

# Parameters
seq_len = 2  # example sequence length

# Create datasets
train_dataset = WindPowerDataset(train_ori, seq_len)
test_dataset = WindPowerDataset(test_ori, seq_len)

TypeError: 'DataFrame' object is not callable